In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import classification_report, precision_score, recall_score, confusion_matrix, roc_curve, auc, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
# display confusion matrix:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
#from tune_sklearn import TuneSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import itertools
from sklearn.preprocessing import LabelBinarizer
import imblearn
import lightgbm as lgb
targs = ['target_question','target_discarded','target_subsection','target_section']


In [16]:
%%time
file_path = 'GT_IPI.parquet'
df = pd.read_parquet(file_path, engine='pyarrow')

CPU times: total: 16.3 s
Wall time: 6.62 s


In [20]:
df[df['css_pk']=='3825451'].to_csv('bad guy.csv')

In [3]:
indices_df = pd.read_csv('unique_indexes.csv')
indices = indices_df['unique_index'].tolist()
filtered_df = df[df.index.isin(indices)]

# If you want to reset the index after filtering (optional)
filtered_df.reset_index(drop=True, inplace=True)
df = filtered_df

In [4]:
bool_column = df.apply({0,1}.issuperset)
bool_column_list = bool_column[bool_column == True].index.to_list()
style_counts_by_target = df.groupby('target')['style'].value_counts()
# too many style value in the column style
style_counts_by_target = df.groupby('target')['style'].value_counts()

# Convert the result to a DataFrame
style_counts_by_target_df = style_counts_by_target.to_frame(name='Count').reset_index()

# Initialize an empty list to hold the top 10 terms for each target
top_15_per_target = []

# Calculate the percentage representation and get the top 15 terms for each 'target'
for target in style_counts_by_target_df['target'].unique():
    # Filter the DataFrame for the current target
    target_df = style_counts_by_target_df[style_counts_by_target_df['target'] == target]
    
    # Calculate the total number of terms for the current target
    total_terms = target_df['Count'].sum()
    
    # Calculate the percentage representation of each term
    target_df['Percentage'] = (target_df['Count'] / total_terms) * 100
    
    # Get the top 15 terms
    top_15 = target_df.head(15)
    
    # Append the result to the list
    top_15_per_target.append(top_15)

# Concatenate all top 10 DataFrames into a single DataFrame
combined_top_15 = pd.concat(top_15_per_target)

# Display the unique 'style' values in the combined DataFrame
unique_styles = combined_top_15['style'].unique()
## manually remove title from the list as it is duplicated
#unique_styles = np.delete(unique_styles, np.where(unique_styles == 'title'))
print("Unique 'style' values in the top 15 per target:")
print(unique_styles)
# Transform the 'style' column to 'None' if the current value isn't found in unique_styles
df['style'] = df['style'].apply(lambda x: x if x in unique_styles else 'None')
# Keep a copy of the original 'target' column
original_target = df['target'].copy()

# Apply one-hot encoding to the specified columns
#df_encoded = pd.get_dummies(df, columns=['target', 'begins_with', 'style'], prefix=['target', 'begins_with', 'style'])
df_encoded = pd.get_dummies(df,drop_first = True
                            , columns=['begins_with','para_foll_depth_ind','para_foll_size_ind', 'para_prec_depth_ind', 'para_prec_size_ind', 'style'] + bool_column_list\
                            , prefix=['begins_with','para_foll_depth_ind','para_foll_size_ind', 'para_prec_depth_ind', 'para_prec_size_ind', 'style'] + bool_column_list)
df_encoded = pd.get_dummies(df_encoded
                            , columns=['target']\
                            , prefix=['target'] )

# Add the original 'target' column back to the encoded DataFrame
df_encoded['target'] = original_target



C:\Users\wyatt\AppData\Local\Temp\ipykernel_4724\793476661.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['Percentage'] = (target_df['Count'] / total_terms) * 100
C:\Users\wyatt\AppData\Local\Temp\ipykernel_4724\793476661.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['Percentage'] = (target_df['Count'] / total_terms) * 100
C:\Users\wyatt\AppData\Local\Temp\ipykernel_4724\793476661.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Unique 'style' values in the top 15 per target:
['default' 'listparagraph' 'bodytext' 'tableparagraph' 'normal' 'heading2'
 'toc2' 'toc1' 'heading1' 'nospacing' 'heading3' 'normalweb' 'tabletext'
 'toc3' 'listnumber' 'question' 'dvquestion' 'paragraphedeliste'
 'esisquestion' 'title' 'heading4' 'heading10' 'paragraph' 'titre1'
 'dvsubcategory' 'rfpheading3' 'himheading4bodybold' 'heading20']


In [5]:
customers_with_less_than2_csspk = [
    x for x in df_encoded['customer_pk'].unique() 
    if len(df_encoded[df_encoded['customer_pk'] == x]['css_pk'].unique()) < 2
]

filtered_df = df_encoded[~df_encoded['customer_pk'].isin(customers_with_less_than2_csspk)]

In [6]:
#array(['question', 'discarded', 'subsection', 'section'],
exclude = ['lang_num_sents','css_pk','customer_pk','html_pk','id']
#top_15_columns
top_15_columns = []
columns_to_exclude = exclude + top_15_columns

In [8]:
from sklearn.model_selection import GroupShuffleSplit
# Lists to store the different train and test dataframes
r_state = [1, 2, 3, 4, 5]
train_df_dict = dict()
test_df_dict = dict()

# Generate 5 different train/test splits
for r in r_state:
    gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=10*r)
    train_list = []
    test_list = []

    # Perform train-test split within each group
    for i in filtered_df['customer_pk'].unique():
        X = filtered_df.loc[filtered_df['customer_pk'] == i]
        train_index, test_index = next(gss.split(X, groups=X['css_pk']))
        
        # Append the subset data to the respective list
        train_list.append(X.iloc[train_index])
        test_list.append(X.iloc[test_index])

    # Concatenate the lists to create final train and test DataFrames
    train_df_dict[r] = pd.concat(train_list, ignore_index=True)
    test_df_dict[r] = pd.concat(test_list, ignore_index=True)

# Check the length of one of the train and test sets
print(f"Train DataFrame length for r=1: {len(train_df_dict[1])}")
print(f"Test DataFrame length for r=1: {len(test_df_dict[1])}")

Train DataFrame length for r=1: 2438929
Test DataFrame length for r=1: 680281


In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

def multi_pred_stats(predicts_list):
    # Extract true labels from each prediction list
    true_question = [subarray[1] for subarray in predicts_list[0]]
    true_discarded = [subarray[1] for subarray in predicts_list[1]]
    true_subsection = [subarray[1] for subarray in predicts_list[2]]
    true_section = [subarray[1] for subarray in predicts_list[3]]
    
    # Stack arrays and determine the maximum indices
    stacked_arrays = np.vstack((true_question, true_discarded, true_section, true_subsection))
    label_names = ['question', 'discarded', 'section', 'subsection']
    max_indices = np.argmax(stacked_arrays, axis=0)
    result = [label_names[i] for i in max_indices]
    
    # Calculate classification report for the combined prediction
    y_true_overall = test_df['target']
    y_pred_overall = result
    overall_class_report = classification_report(y_true_overall, y_pred_overall)
    print(overall_class_report)
    return overall_class_report


In [23]:
import lightgbm as lgb
import imblearn
from imblearn.over_sampling import SMOTE, ADASYN
# Initialize lists to store reports and features
def train_test(target, sampling_tech='none', lgb_params=None):
    if lgb_params is None:
        lgb_params = {}

    X_train = train_df.drop(columns=columns_to_exclude + ['target'] + targs)
    y_train = train_df[f'target_{target}']
    X_test = test_df.drop(columns=columns_to_exclude + ['target'] + targs)
    y_test = test_df[f'target_{target}']

    if sampling_tech == 'under':
        rus = imblearn.under_sampling.RandomUnderSampler(random_state=420)
    elif sampling_tech == 'over':
        rus = imblearn.over_sampling.RandomOverSampler(random_state=420)
    elif sampling_tech == 'smote':
        rus = SMOTE(random_state=420)
    elif sampling_tech == 'adasyn':
        rus = ADASYN(random_state=420)
    
    if sampling_tech in ['over', 'under', 'smote', 'adasyn']:
        X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
        X_train = X_train_resampled
        y_train = y_train_resampled

    model = lgb.LGBMClassifier(**lgb_params)
    model.fit(X_train, y_train)
    predicts = model.predict_proba(X_test)

    return predicts, model

params_s = {'bagging_fraction': 0.3633904999185218, 'feature_fraction': 1.0, 'lambda_l1': 3.574560893629339, 'lambda_l2': 9.818056164657532, 'learning_rate': 0.4851255385542588, 'max_depth': 12, 'min_child_samples': 81, 'min_child_weight': 12.168209299428229, 'min_split_gain': 0.05264829965012845, 'n_estimators': 506, 'num_leaves': 16}
params_d = {'bagging_fraction': 0.6243816730492119, 'feature_fraction': 0.45028179026459975, 'lambda_l1': 1.1971042837442278, 'lambda_l2': 4.297245941070125, 'learning_rate': 0.5981568321835795, 'max_depth': 9, 'min_child_samples': 83, 'min_child_weight': 12.608881674364874, 'min_split_gain': 0.5222560899891926, 'n_estimators': 506, 'num_leaves': 52}
params_q = {'bagging_fraction': 0.9702107536403743, 'feature_fraction': 0.8341182143924175, 'lambda_l1': 2.1233911067827616, 'lambda_l2': 1.8182496720710062, 'learning_rate': 0.15488956278421273, 'max_depth': 8, 'min_child_samples': 57, 'min_child_weight': 8.644580922655893, 'min_split_gain': 0.2912291401980419, 'n_estimators': 612, 'num_leaves': 14}
params_ss = {'bagging_fraction': 0.09154053332582732, 'feature_fraction': 0.8620892013062822, 'lambda_l1': 2.979058792848589, 'lambda_l2': 6.228517664360759, 'learning_rate': 0.36057259624318583, 'max_depth': 15, 'min_child_samples': 86, 'min_child_weight': 11.202870028573141, 'min_split_gain': 0.456877355938568, 'n_estimators': 501, 'num_leaves': 20}

In [20]:
reports1 = []

for r in r_state:
    print(f'run {r}')
    train_df = train_df_dict[r]
    test_df = test_df_dict[r]
    z = multi_pred_stats([train_test('question')[0],train_test('discarded')[0],train_test('subsection')[0],train_test('section')[0]])
    reports1.append(z)
    print(z)

run 1
[LightGBM] [Info] Number of positive: 1135215, number of negative: 1303714
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.241390 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8503
[LightGBM] [Info] Number of data points in the train set: 2438929, number of used features: 112
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.465456 -> initscore=-0.138395
[LightGBM] [Info] Start training from score -0.138395
[LightGBM] [Info] Number of positive: 1221432, number of negative: 1217497
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.231718 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8503
[LightGBM] [Info] Number of data points in the train set: 2438929, number of used features: 11

In [22]:
for res in reports1:
    print(res)

              precision    recall  f1-score   support

   discarded       0.78      0.82      0.80    343880
    question       0.80      0.79      0.80    313673
     section       0.53      0.20      0.29     12516
  subsection       0.56      0.06      0.11     10212

    accuracy                           0.79    680281
   macro avg       0.67      0.47      0.50    680281
weighted avg       0.78      0.79      0.78    680281

              precision    recall  f1-score   support

   discarded       0.78      0.84      0.81    330279
    question       0.82      0.80      0.81    310293
     section       0.54      0.19      0.28     12745
  subsection       0.49      0.05      0.09      9174

    accuracy                           0.80    662491
   macro avg       0.66      0.47      0.50    662491
weighted avg       0.79      0.80      0.79    662491

              precision    recall  f1-score   support

   discarded       0.77      0.83      0.80    333038
    question       0.

In [24]:
reports2 = []

for r in r_state:
    print(f'run {r}')
    train_df = train_df_dict[r]
    test_df = test_df_dict[r]
    z = multi_pred_stats([train_test('question')[0],train_test('discarded')[0],train_test('subsection','over')[0],train_test('section')[0]])
    reports2.append(z)
    print(z)

run 1
[LightGBM] [Info] Number of positive: 1135215, number of negative: 1303714
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.224365 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8503
[LightGBM] [Info] Number of data points in the train set: 2438929, number of used features: 112
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.465456 -> initscore=-0.138395
[LightGBM] [Info] Start training from score -0.138395
[LightGBM] [Info] Number of positive: 1221432, number of negative: 1217497
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.231217 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8503
[LightGBM] [Info] Number of data points in the train set: 2438929, number of used features: 11

In [25]:
for res in reports2:
    print(res)

              precision    recall  f1-score   support

   discarded       0.81      0.74      0.77    343880
    question       0.81      0.78      0.79    313673
     section       0.65      0.02      0.03     12516
  subsection       0.11      0.70      0.20     10212

    accuracy                           0.75    680281
   macro avg       0.60      0.56      0.45    680281
weighted avg       0.80      0.75      0.76    680281

              precision    recall  f1-score   support

   discarded       0.81      0.76      0.78    330279
    question       0.83      0.79      0.81    310293
     section       0.76      0.01      0.03     12745
  subsection       0.11      0.68      0.18      9174

    accuracy                           0.76    662491
   macro avg       0.63      0.56      0.45    662491
weighted avg       0.81      0.76      0.77    662491

              precision    recall  f1-score   support

   discarded       0.80      0.75      0.77    333038
    question       0.

In [24]:
reports3 = []

for r in r_state:
    print(f'run {r}')
    train_df = train_df_dict[r]
    test_df = test_df_dict[r]
    z = multi_pred_stats([train_test('question','none',params_q)[0],train_test('discarded','under',params_d)[0],train_test('subsection','over',params_ss)[0],train_test('section','over',params_s)[0]])
    reports3.append(z)
    print(z)

run 1
[LightGBM] [Warning] feature_fraction is set=0.8341182143924175, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8341182143924175
[LightGBM] [Warning] lambda_l1 is set=2.1233911067827616, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.1233911067827616
[LightGBM] [Warning] lambda_l2 is set=1.8182496720710062, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8182496720710062
[LightGBM] [Warning] bagging_fraction is set=0.9702107536403743, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9702107536403743
[LightGBM] [Warning] feature_fraction is set=0.8341182143924175, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8341182143924175
[LightGBM] [Warning] lambda_l1 is set=2.1233911067827616, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.1233911067827616
[LightGBM] [Warning] lambda_l2 is set=1.8182496720710062, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8182496720710062
[Ligh

In [25]:
for res in reports3:
    print(res)

              precision    recall  f1-score   support

   discarded       0.82      0.75      0.78    343880
    question       0.82      0.79      0.81    313673
     section       0.27      0.65      0.39     12516
  subsection       0.13      0.42      0.20     10212

    accuracy                           0.76    680281
   macro avg       0.51      0.65      0.54    680281
weighted avg       0.80      0.76      0.78    680281

              precision    recall  f1-score   support

   discarded       0.82      0.76      0.79    330279
    question       0.84      0.80      0.82    310293
     section       0.29      0.64      0.40     12745
  subsection       0.12      0.42      0.19      9174

    accuracy                           0.77    662491
   macro avg       0.52      0.66      0.55    662491
weighted avg       0.81      0.77      0.79    662491

              precision    recall  f1-score   support

   discarded       0.81      0.75      0.78    333038
    question       0.

In [21]:
reports7 = []

for r in r_state:
    print(f'run {r}')
    train_df = train_df_dict[r]
    test_df = test_df_dict[r]
    z = multi_pred_stats([train_test('question','none')[0],train_test('discarded','under')[0],train_test('subsection','over')[0],train_test('section','over')[0]])
    reports7.append(z)
    print(z)
for res in reports7:
    print(res)

run 1
[LightGBM] [Info] Number of positive: 1135215, number of negative: 1303714
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.246543 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8503
[LightGBM] [Info] Number of data points in the train set: 2438929, number of used features: 112
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.465456 -> initscore=-0.138395
[LightGBM] [Info] Start training from score -0.138395
[LightGBM] [Info] Number of positive: 1217497, number of negative: 1217497
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.371928 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8491
[LightGBM] [Info] Number of data points in the train set: 2434994, number of used features: 11

In [31]:
reports4 = []

for r in r_state:
    print(f'run {r}')
    train_df = train_df_dict[r]
    test_df = test_df_dict[r]
    z = multi_pred_stats([train_test('question','none',params_q)[0],train_test('discarded','under',params_d)[0],train_test('subsection','none',params_ss)[0],train_test('section','over',params_s)[0]])
    reports4.append(z)
    print(z)

run 1
[LightGBM] [Warning] feature_fraction is set=0.820049231809358, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.820049231809358
[LightGBM] [Warning] lambda_l1 is set=0.35398913607890153, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.35398913607890153
[LightGBM] [Warning] lambda_l2 is set=2.413763938888623, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.413763938888623
[LightGBM] [Warning] bagging_fraction is set=0.7939521214604877, subsample=0.3170686952817686 will be ignored. Current value: bagging_fraction=0.7939521214604877
[LightGBM] [Warning] feature_fraction is set=0.820049231809358, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.820049231809358
[LightGBM] [Warning] lambda_l1 is set=0.35398913607890153, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.35398913607890153
[LightGBM] [Warning] lambda_l2 is set=2.413763938888623, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.4137639388

In [32]:
for res in reports4:
    print(res)

              precision    recall  f1-score   support

   discarded       0.86      0.70      0.77    343880
    question       0.76      0.87      0.82    313673
     section       0.25      0.80      0.38     12516
  subsection       0.70      0.01      0.01     10212

    accuracy                           0.77    680281
   macro avg       0.64      0.60      0.50    680281
weighted avg       0.80      0.77      0.77    680281

              precision    recall  f1-score   support

   discarded       0.86      0.72      0.78    330279
    question       0.78      0.88      0.83    310293
     section       0.26      0.79      0.39     12745
  subsection       0.81      0.01      0.02      9174

    accuracy                           0.78    662491
   macro avg       0.68      0.60      0.50    662491
weighted avg       0.81      0.78      0.79    662491

              precision    recall  f1-score   support

   discarded       0.85      0.70      0.77    333038
    question       0.